In [4]:
import pandas as pd
import xlwings as xw

In [5]:
# make sure you save file first and unfilter all tables
wb = xw.Workbook('U:/SQL/PUBLIC/PORTAL/NA_sector_itsys_mapping/na_sector_itsys_template.xlsx')

In [6]:
target_sht = 'Sheet1'

In [119]:
# get IT system mapping by client

In [7]:
sys_data = xw.Range('client_sys_lookup','A5').table.value  # refresh client_sys_lookup tab first
sys_df = pd.DataFrame(data=sys_data[1:],columns=sys_data[0])
sys_df = sys_df[sys_df.in_template != 'None'] # exclude clients that aren't in template provided
sys_df = sys_df.iloc[:, 1:3] # filter to client/sys
sys_df.columns = ['CLIENT CODE', 'Warehouse Management System'] # update column names
sys_df = sys_df[sys_df['Warehouse Management System'] != 'GWS(LMS)'] # remove clients with GWS(LMS)

In [8]:
# get data on template sheet provided by BIA

In [9]:
t_data = xw.Range('Template', 'A1').table.value
t_df = pd.DataFrame(data=t_data[1:], columns=t_data[0])
t_df = t_df.drop('Warehouse Management System',1) # drop their itsys and use what we have

In [10]:
# merge our data with their data and conserve structure

In [11]:
merged_df = sys_df.merge(t_df, how='inner', on='CLIENT CODE')
merged_df = merged_df.append(t_df[t_df.client_sys_count == 0]) # appends clients that don't appear in client_sys_count
col_order =['REPORTING BUSINESS UNIT ID'
        ,'REPORTING BUSINESS UNIT'
        ,'REGION ID'
        ,'REGION NAME'
        ,'COUNTRY CODE'
        ,'COUNTRY NAME'
        ,'CLIENT CODE'
        ,'CLIENT NAME'
        ,'SECTOR CODE'
        ,'SECTOR DESC'
        ,'BUSINESS SEGMENT'
        ,'Warehouse Management System'
        ,'Transportation Management System'
        ,'Repair & Return Flag'
        ,'QA Flag']
merged_df = merged_df.reindex(columns=col_order) # set column order

In [12]:
# get transportation it sys data

In [13]:
details_data = xw.Range('it_sys_details', 'B11').table.value
details_df = pd.DataFrame(data=details_data[1:], columns=details_data[0])
details_df = details_df[['client_number','Warehouse Mgmt System', 'Transportation Mgmt System']]
# remove client-locations with nothing under mgmt/trans systems
details_df = details_df[(details_df['Warehouse Mgmt System'] != '')&(details_df['Transportation Mgmt System'] != '')]

In [14]:
# categorize trans it sys

In [15]:
trans_mgmt_sys_adj = []
for sys in details_df['Transportation Mgmt System']:
    if sys == 'ETMS' or sys == 'OTM Transportation Mgmt':
        trans_mgmt_sys_adj.append('ETMS')
    elif sys == 'RTM':
        trans_mgmt_sys_adj.append('RTM')
    else:
        trans_mgmt_sys_adj.append("")
details_df['trans_mgmt_sys_adj'] = trans_mgmt_sys_adj

In [16]:
# conserve data structure and drop duplicates to circumvent multiplying rows in output

In [17]:
details_df = details_df.drop('Transportation Mgmt System', 1)
details_df = details_df.rename(columns={"client_number":"CLIENT CODE"
                                        ,"trans_mgmt_sys_adj":"Transportation Management System"
                                        ,"Warehouse Mgmt System":"Warehouse Management System"})
details_df = details_df.drop_duplicates()

In [18]:
# merge trans it sys data with the rest of our data
# join on client code AND warehouse mgmt sys so that trans it sys is mapped accurately

In [19]:
merged_df = merged_df.drop('Transportation Management System',1)
merged_df = merged_df.merge(details_df, how='left', on=['CLIENT CODE', 'Warehouse Management System'])
merged_df = merged_df.reindex(columns=col_order)

In [20]:
# paste to target_sht

In [21]:
xw.Sheet(target_sht).clear_contents()
xw.Range(target_sht,'A1').value = merged_df.set_index('REPORTING BUSINESS UNIT ID')

In [22]:
# check that RTM output is close to what we have in it_sys_details tab
#d2 = details_df[details_df['Transportation Management System'] == 'RTM']
#d2 = d2.drop_duplicates()
#xw.Range('Sheet2', 'K1').value = d2

In [23]:
# manually delete the extra rows for the following clients
del_df = details_df.merge(details_df, how='left', on=['CLIENT CODE', 'Warehouse Management System'],sort='CLIENT CODE')
del_df[del_df['Transportation Management System_x'] != del_df['Transportation Management System_y']]

,CLIENT CODE,Warehouse Management System,Transportation Management System_x,Transportation Management System_y
10,0000403,PkMS(GWS),ETMS,
11,0000403,PkMS(GWS),,ETMS
148,0005571,LMS,ETMS,
149,0005571,LMS,,ETMS
164,0005861,EXCEED,,ETMS
165,0005861,EXCEED,ETMS,
